# 04.4- Agents



Let's take a look at agents, and expand the capabilities of LLMs.

### AgentType

There are several types of agents, each optimized for what we need. An agent can store past chat information or can enforce each call to the LLM as an independent one.  
It is up to you to decide what you need in your future projects, but here we will focus on agents capable of maintaining a chat.

### Tools

The `tools` are a set of options one uses to give further capabilities to the agent: parsing wikipedia, using google for searches, etc..  
Several models are trained with a 1-2 year delay in data, like chatgpt-3.5. It was only trained with data up to 2022. Some tools may help to overcome this limitation with active searches. 

In [ ]:
#!pip install numexpr
#!pip install langchainhub
#!pip install langchain_openai

In [2]:
from langchain.agents import AgentType # The type of agent
from langchain.memory import ConversationBufferMemory # Memory
from langchain.agents import initialize_agent 
from langchain.agents import load_tools #Additional tools
from langchain_openai import ChatOpenAI # Our LLM
from IPython.display import Markdown #Pretty display

In [3]:
# Let's create a memory instance for our agent
memory = ConversationBufferMemory(memory_key="chat_history")

In [4]:
llm=ChatOpenAI(temperature=0.1)
tools = load_tools(["llm-math"], llm=llm) 
agent_chain = initialize_agent(tools, 
                               llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, 
                               memory=memory)

C:\Users\luisg\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
answer = agent_chain.invoke(input="What are some good Portuguese food recipes?", handle_parsing_errors=True)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Some popular Portuguese food recipes include Bacalhau à Brás (salted cod with potatoes and eggs), Caldo Verde (kale soup), and Pastéis de Nata (custard tarts). Would you like more information on any specific recipe?

> Finished chain.


In [6]:
Markdown(answer['output'])

Some popular Portuguese food recipes include Bacalhau à Brás (salted cod with potatoes and eggs), Caldo Verde (kale soup), and Pastéis de Nata (custard tarts). Would you like more information on any specific recipe?

In [7]:
answer = agent_chain.run("Which one of those dishes has the least fat?")

C:\Users\luisg\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Fat content of Bacalhau à Brás, Caldo Verde, and Pastéis de Nata
Observation: Answer: 43
Thought:Do I need to use a tool? No
AI: The dish with the least fat content among Bacalhau à Brás, Caldo Verde, and Pastéis de Nata is Caldo Verde, which is a kale soup.

> Finished chain.


Now the history of our conversation is stored and you can build upon what you have chatted.

In [8]:
agent_chain.run("Give me a grocery shopping list to make that dish")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Calculate grocery shopping list for Caldo Verde recipe
Observation: Answer: 6.5
Thought:Do I need to use a tool? No
AI: Here is a grocery shopping list for making Caldo Verde:
- Kale
- Potatoes
- Onion
- Garlic
- Chorizo (optional)
- Olive oil
- Salt
- Water

These are the main ingredients you will need to make a delicious Caldo Verde soup. Enjoy your cooking!

> Finished chain.


'Here is a grocery shopping list for making Caldo Verde:\n- Kale\n- Potatoes\n- Onion\n- Garlic\n- Chorizo (optional)\n- Olive oil\n- Salt\n- Water\n\nThese are the main ingredients you will need to make a delicious Caldo Verde soup. Enjoy your cooking!'

---
### Other applications: Analyzing documents

The Langchain framework has a lot of code already specialized in tasks meant to be done over and over.  
In this example we go over a text file that we have seen before, the play from William Shakespeare.

In [9]:
import requests

# Let's get our Shakespeare back
response = requests.get("https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

# There is a size limit for inputs
shakespeare = response.text[0:10000]

In [10]:
from langchain.chains import AnalyzeDocumentChain

In [12]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm)
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [13]:
qa_document_chain.run(
    input_document=shakespeare,
    question="What is this work?",
)

'The text provided is an excerpt from the play "Coriolanus" by William Shakespeare. It is a tragedy believed to have been written between 1605 and 1608.'

In [14]:
qa_document_chain.run(
    input_document=shakespeare,
    question="Describe the main protagonist of this work.",
)

'The main protagonist of this work is Caius Marcius, also known as Coriolanus. He is a Roman general who is initially portrayed as proud and disdainful of the common people. He is a skilled warrior and is respected for his military prowess. However, his arrogance and contempt for the lower classes lead to conflict and ultimately his downfall.'

---
### PDF documents

In [ ]:
#!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# Windows
#!cURL https://media.wizards.com/images/magic/resources/rules/EN_MTGM14_PrintedRulebook_LR.pdf >> EN_MTGM14_PrintedRulebook_LR.pdf

In [ ]:
# MAC/Linux
#!wget https://media.wizards.com/images/magic/resources/rules/EN_MTGM14_PrintedRulebook_LR.pdf

In [ ]:
loader = PyPDFLoader("EN_MTGM14_PrintedRulebook_LR.pdf")
pages = loader.load_and_split()

In [ ]:
pages[10]

A PDF is usually text+images. Ideally, you send both the images and the the text to the model for analysis.  
But you need a multi-modal model for that, which we will not cover due to time constraints.  
Here, we are just going to get the text and run some simple queries.

In [ ]:
pages[10].to_json()['kwargs']['page_content']

In [ ]:
text = []

# Append all pages on a list and then join all list elements
for page in pages:
    text.append(page.to_json()['kwargs']['page_content'])

all_text = '\n'.join(text)

In [ ]:
# Number of pages, number of characters
len(text), len(all_text)

The document is too big for our model, we need to focus on some pages only.

In [ ]:
text = []

# Append all pages on a list and then join all list elements
for pagenumber in range(2,6):
    text.append(pages[pagenumber].to_json()['kwargs']['page_content'])

all_text = '\n'.join(text)

In [ ]:
# Number of pages, number of characters
len(text), len(all_text)

In [ ]:
text

In [ ]:
answer = qa_document_chain.run(input_document=all_text,
                               question="What are the parts of a card?",)

Markdown(answer)

To analyse more content, one must use models capable of handling more tokens.